<a href="https://colab.research.google.com/github/arpitav03/Data-Pipeline-Practice-Basics/blob/main/Comparison___Pandas_Polars_PySpark_Datapipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install pyspark[sql]

## **Using Pandas**

In [9]:
import pandas as pd
flights = "flights.parquet"

In [10]:
df = pd.read_parquet(flights)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['Airline']

**Aggregations and saving in a new file**

In [20]:
df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean","sum","max"]
)
df_agg.reset_index()
df_agg.to_parquet("temp_pandas.parquet")

In [ ]:
pd.read_parquet("temp_pandas.parquet")

### **Using Polars**

In [24]:
import polars as pl
df_polars = (pl.scan_parquet(flights).groupby(['Airline','Year'])\
                .agg(
                    [pl.col("DepDelayMinutes").mean().alias("avg_dep_delay"),
                     pl.col("DepDelayMinutes").sum().alias("sum_dep_delay"),
                     pl.col("DepDelayMinutes").max().alias("max_dep_delay"),
                     pl.col("ArrDelayMinutes").mean().alias("avg_arr_delay"),
                     pl.col("ArrDelayMinutes").sum().alias("sum_arr_delay"),
                     pl.col("ArrDelayMinutes").max().alias("max_arr_delay")])
).collect()

df_polars.write_parquet('temp_polars.parquet')

In [ ]:
df_polars.head

## **Using Pyspark**

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg,max,sum

#Create SparkSession

spark = SparkSession.builder.master("local[1]").appName("airline-example").getOrCreate()
flights = "flights.parquet"

In [ ]:
df_spark = spark.read.parquet(flights)
df_spark_agg = df_spark.groupby("Airline", "Year").agg(
                    avg("DepDelayMinutes").alias("avg_dep_delay"),
                    sum("DepDelayMinutes").alias("sum_dep_delay"),
                    max("DepDelayMinutes").alias("max_dep_delay"),
                    avg("ArrDelayMinutes").alias("avg_arr_delay"),
                    sum("ArrDelayMinutes").alias("sum_arr_delay"),
                    max("ArrDelayMinutes").alias("max_arr_delay"))
df_spark_agg.write.mode('overwrite').parquet('temp.parquet')
pd.read_parquet('temp.parquet')